# Docker "Swarm Mode" Lab

Based on Mario's gist here: https://gist.github.com/l0rd/5186cc80f8f26dc7e9490abca4405830

# Requirements
- Docker 1.12
- Docker machine
- Virtualbox

# Create 3 nodes for your swarm cluster (one master and 2 slaves)

We will create 3 nodes using docker-machine/virtualbox.

#### NOTE:
You may see errors as below, and an error reported by "docker-machine ls", leave some time for the swmaster1 to settle.

In the meantime you can go ahead and create the 2 swnode's below.

In [1]:
MACHINE_DRIVER=virtualbox

. ../NB_bash_functions.rc

Notebook started at Sat May 13 20:02:48 CEST 2017   [1494698568]


In [2]:
# . ~/.docker.rc

In [3]:
MACHINE_DRIVER=virtualbox

#MACHINE_DRIVER=digitalocean
#MACHINE_DRIVER=azure
#MACHINE_DRIVER=google

Let's cleanup any remaining machines, if they exist already ..

In [4]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls


NAME        ACTIVE   DRIVER       STATE     URL   SWARM   DOCKER    ERRORS
swmaster1   -        virtualbox   Stopped                 Unknown   


In [5]:
docker-machine ls --filter name=sw --format "{{.Name}}: {{.DriverName}}"

swmaster1: virtualbox


In [6]:
NB_cleanup_swarm_machines;

docker-machine rm -f swmaster1
About to remove swmaster1
Successfully removed swmaster1


In [7]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls


NAME   ACTIVE   DRIVER   STATE   URL   SWARM   DOCKER   ERRORS


Now let's create our new nodes

**NOTE:** You may see some worrying messages from "*docker-machine ls*" but wait a minute or two until the node creation is complete

In [8]:
NB_create_swarm_machine_m1

Running command <docker-machine create -d virtualbox swmaster1> ...
Running pre-create checks...
Creating machine...
(swmaster1) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swmaster1/boot2docker.iso...
(swmaster1) Creating VirtualBox VM...
(swmaster1) Creating SSH key...
(swmaster1) Starting the VM...
(swmaster1) Check network to re-create if needed...
(swmaster1) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env swmaster1
Took 106 secs [0h01m4

#### NOTE: Temporary errors from "*docker-machine ls*"

From the command-line you can see the progress of machine creation, using
    ```$ docker-machine ls```
    
You may see errors during creation of machines before ssh connectivity is established and before docker host is started.

![](images/docker-machine-errors.png)

After creation of a node you should see something like the following from "*docker-machine ls*"

In [9]:
docker-machine ls

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.114:2376           v17.05.0-ce   


We can also see the VM from virtualbox point of view (if this is the driver we are using):

In [10]:
VBoxManage list runningvms | grep sw

"swmaster1" {4e7024ab-4f9f-4dbe-879e-cead2545e8ca}


In [11]:
NB_create_swarm_machine_1

Running command <docker-machine create -d virtualbox swnode1> ...
Running pre-create checks...
Creating machine...
(swnode1) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swnode1/boot2docker.iso...
(swnode1) Creating VirtualBox VM...
(swnode1) Creating SSH key...
(swnode1) Starting the VM...
(swnode1) Check network to re-create if needed...
(swnode1) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env swnode1
Took 123 secs [0h02m03]


In [12]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

"swmaster1" {4e7024ab-4f9f-4dbe-879e-cead2545e8ca}
"swnode1" {62ce063a-56ef-452f-8d65-4146a02dc469}

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.114:2376           v17.05.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.115:2376           v17.05.0-ce   


In [13]:
NB_create_swarm_machine_2

Running command <docker-machine create -d virtualbox swnode2> ...
Running pre-create checks...
Creating machine...
(swnode2) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swnode2/boot2docker.iso...
(swnode2) Creating VirtualBox VM...
(swnode2) Creating SSH key...
(swnode2) Starting the VM...
(swnode2) Check network to re-create if needed...
(swnode2) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env swnode2
Took 113 secs [0h01m53]


In [14]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

"swmaster1" {4e7024ab-4f9f-4dbe-879e-cead2545e8ca}
"swnode1" {62ce063a-56ef-452f-8d65-4146a02dc469}
"swnode2" {12e313bd-0d91-4b4e-98d2-32523757c759}

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.114:2376           v17.05.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.115:2376           v17.05.0-ce   
swnode2     -        virtualbox   Running   tcp://192.168.99.116:2376           v17.05.0-ce   


In [15]:
NB_check_3_machines_running

# swarm init

Now that we have 3 nodes available, we will initialize our Swarm Cluster with 1 master node.

### Directing the docker client to a particular nodes' docker daemon
**NOTE**: See that we precede all docker commands with $(docker-machine config NODE) where node is the name of the node to which we want our docker client to connect to.  This command returns the parameters to direct our client to the appropriate node.  Run alone this produces:

In [16]:
docker-machine config swmaster1

--tlsverify
--tlscacert="/home/mjb/.docker/machine/machines/swmaster1/ca.pem"
--tlscert="/home/mjb/.docker/machine/machines/swmaster1/cert.pem"
--tlskey="/home/mjb/.docker/machine/machines/swmaster1/key.pem"
-H=tcp://192.168.99.114:2376


We can also obtain these values as environment variables

In [17]:
docker-machine env swmaster1

export DOCKER_TLS_VERIFY="1"
export DOCKER_HOST="tcp://192.168.99.114:2376"
export DOCKER_CERT_PATH="/home/mjb/.docker/machine/machines/swmaster1"
export DOCKER_MACHINE_NAME="swmaster1"
# Run this command to configure your shell: 
# eval $(docker-machine env swmaster1)


Including these parameters on the docker command line will connect the client to the docker daemon running on node '*swmaster1*'.

### Networks before creation of swarm cluster
Before going further let's look at the networks on your machine.

Later, we'll see how a new network is created once the swarm cluster has been created.

In [18]:
docker $(docker-machine config swmaster1) network ls

NETWORK ID          NAME                DRIVER              SCOPE
4a285364bdf1        bridge              bridge              local
f037776427e3        host                host                local
dacef2ccdca4        none                null                local


Now let's identify the ip address of our master node.

We can see this through config or ip commands of docker-machine as shown below.

In [19]:
docker-machine ip swmaster1

192.168.99.114


We could then provide the above ip address as parameter to --advertise-addr when initializing the swarm.

However, it is quite convenient to run the above commands embedded, as below, as arguments to the swarm init command.

docker-machine config swmaster1 provides the parameters to use when connecting to the appropriate docker engine for our machine "swmaster1".

The following command will run swarm init to generate the cluster with 'swmaster1' as the Master node.
You should see output similar to the below:

In [20]:
docker $(docker-machine config swmaster1) info | grep Swarm:

Swarm: inactive


In [21]:
docker $(docker-machine config swmaster1) swarm init --advertise-addr $(docker-machine ip swmaster1)

Swarm initialized: current node (m5i0938m512yeqtj2vgwoss38) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join \
    --token SWMTKN-1-1vpttsk2y0jsvnk7emlmm60a02ngiqz2vdje27smtwg0rmb6wr-12ctnevv31rcuezgjj50dupvf \
    192.168.99.114:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



A docker info should now show "Swarm: active" as below:

In [22]:
docker $(docker-machine config swmaster1) info | grep Swarm:

Swarm: active


In [23]:
docker $(docker-machine config swmaster1) info

Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: 17.05.0-ce
Storage Driver: aufs
 Root Dir: /mnt/sda1/var/lib/docker/aufs
 Backing Filesystem: extfs
 Dirs: 0
 Dirperm1 Supported: true
Logging Driver: json-file
Cgroup Driver: cgroupfs
Plugins: 
 Volume: local
 Network: bridge host macvlan null overlay
Swarm: active
 NodeID: m5i0938m512yeqtj2vgwoss38
 Is Manager: true
 ClusterID: d5saeqg118afxdm7n8h8w6qf6
 Managers: 1
 Nodes: 1
 Orchestration:
  Task History Retention Limit: 5
 Raft:
  Snapshot Interval: 10000
  Number of Old Snapshots to Retain: 0
  Heartbeat Tick: 1
  Election Tick: 3
 Dispatcher:
  Heartbeat Period: 5 seconds
 CA Configuration:
  Expiry Duration: 3 months
 Node Address: 192.168.99.114
 Manager Addresses:
  192.168.99.114:2377
Runtimes: runc
Default Runtime: runc
Init Binary: docker-init
containerd version: 9048e5e50717ea4497b757314bad98ea3763c145
runc version: 9c2d8d184e5da67c95d601382adf14862e4f2228
init version: 949e6fa
Security Options:
 s

If we look at the networks we should now see new networks such as '*ingress*' an overlay network and docker_gwbridge for the swarm cluster.

In [24]:
docker $(docker-machine config swmaster1) network ls

NETWORK ID          NAME                DRIVER              SCOPE
4a285364bdf1        bridge              bridge              local
1db031891ebd        docker_gwbridge     bridge              local
f037776427e3        host                host                local
37hr887vm6z7        ingress             overlay             swarm
dacef2ccdca4        none                null                local


# swarm join

Now we wish to join Master and Worker nodes to our swarm cluster, to do this we need to obtain the token generated during the "swarm init".

Let's save that token to an environment variable as follows:

In [25]:
worker_token=$(docker $(docker-machine config swmaster1) swarm join-token worker -q)

In [26]:
echo $worker_token

SWMTKN-1-1vpttsk2y0jsvnk7emlmm60a02ngiqz2vdje27smtwg0rmb6wr-12ctnevv31rcuezgjj50dupvf


Now we can use this token to join nodes as a worker to this cluster

Note: we could also join nodes as Master, but we have only 3 nodes available.

Let's join swnode1 as a worker node

In [27]:
docker $(docker-machine config swnode1) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

This node joined a swarm as a worker.


Now we can use the same token to join node swnode2 as a worker node


In [28]:
docker $(docker-machine config swnode2) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

This node joined a swarm as a worker.


In [29]:
docker $(docker-machine config swmaster1) node ls

ID                           HOSTNAME   STATUS  AVAILABILITY  MANAGER STATUS
m5i0938m512yeqtj2vgwoss38 *  swmaster1  Ready   Active        Leader
obms842spsey4of2hfp795pa0    swnode2    Ready   Active        
s6eekxu2v21eyysjeelerkfrw    swnode1    Ready   Active        


# start service

First we check for any running services - there should be none in our newly initialized cluster:

In [30]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE


Now we will create a new service based on the docker image mjbright/docker-demo

We will expose this service on port 8080


In [31]:
docker $(docker-machine config swmaster1) service create --replicas 1 --name docker-demo -p 8080:8080 mjbright/docker-demo:20

jusq9inajzd118hqw801qf8t7


Now we list services again and we should see our newly added docker-demo service

In [32]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE
jusq9inajzd1        docker-demo         replicated          0/1                 mjbright/docker-demo:20


In [33]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE
jusq9inajzd1        docker-demo         replicated          0/1                 mjbright/docker-demo:20


In [34]:
NB_docker_loop_until_service_started docker-demo

... waiting for 'docker-demo' replicas to start
... waiting for 'docker-demo' replicas to start
jusq9inajzd1        docker-demo         replicated          1/1                 mjbright/docker-demo:20


it may take a few seconds until the service has a running container (see REPLICAS column) especially if the container image has not yet been downloaded

In [35]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE
jusq9inajzd1        docker-demo         replicated          1/1                 mjbright/docker-demo:20


... and we can look at the service as seen by the cluster:

In [36]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE           ERROR               PORTS
vqet7sf9g2yr        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 5 seconds ago                       


... and we can look at the service on the individual cluster nodes.

Of course as we set replicas to 1 there is only 1 replica of the service for the moment, running on just 1 node of our cluster:

In [37]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE                     COMMAND                  CREATED             STATUS              PORTS               NAMES
bc6e3dda061c        mjbright/docker-demo:20   "/bin/docker-demo ..."   8 seconds ago       Up 7 seconds        8080/tcp            docker-demo.1.vqet7sf9g2yrpwnyr0nr0s3ue


In [38]:
docker $(docker-machine config swnode1) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [39]:
docker $(docker-machine config swnode2) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


# Accessing the service

As we are working remotely we need to create an ssh tunnel through to the swarm cluster to access our service, exposing the port 8080 on your local machine.

We can obtain the ip address of the swarm master node as follows.

In [40]:
docker-machine ip swmaster1

192.168.99.114


Let's set an environment variable with that ip address

In [41]:
MASTERIP=$(docker-machine ip swmaster1)

Then open your web browser at the page http://<MASTERIP>:8080 (replace <MASTERIP> with the appropriate value) and you should see a lovely blue whale, as below:

![](images/docker.png)


Alternatively we can connect using a command-line client (the web server detects if it's a wget or curl request and returns ASCII text instead of an image).

In [42]:
wget -O - http://$MASTERIP:8080

--2017-05-13 20:09:31--  http://192.168.99.114:8080/
Connecting to 192.168.99.114:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/txt]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
          

# scale service

Now we can scale the service to 3 replicas:

In [43]:
docker $(docker-machine config swmaster1) service scale docker-demo=3

docker-demo scaled to 3


In [44]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                      ERROR               PORTS
vqet7sf9g2yr        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 13 seconds ago                                 
eqijlhjhrxd7        docker-demo.2       mjbright/docker-demo:20   swnode1             Running             Preparing less than a second ago                       
hksw0wxtx4il        docker-demo.3       mjbright/docker-demo:20   swnode2             Running             Preparing less than a second ago                       


# rolling-update

Now we will see how we can perform a rolling update.

We initially deployed version 20 of the service, now we will upgrade our whole cluster to version 21


In [45]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE             ERROR               PORTS
vqet7sf9g2yr        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 14 seconds ago                        
eqijlhjhrxd7        docker-demo.2       mjbright/docker-demo:20   swnode1             Running             Preparing 2 seconds ago                       
hksw0wxtx4il        docker-demo.3       mjbright/docker-demo:20   swnode2             Running             Preparing 2 seconds ago                       


In [46]:
docker $(docker-machine config swmaster1) service update --image mjbright/docker-demo:21 docker-demo

docker-demo


In [47]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                     ERROR               PORTS
vqet7sf9g2yr        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 21 seconds ago                                
9zxra8gie7yc        docker-demo.2       mjbright/docker-demo:21   swnode1             Running             Preparing 1 second ago                                
eqijlhjhrxd7         \_ docker-demo.2   mjbright/docker-demo:20   swnode1             Shutdown            Shutdown less than a second ago                       
hksw0wxtx4il        docker-demo.3       mjbright/docker-demo:20   swnode2             Running             Preparing 8 seconds ago                               


Again it may take a few moments until the new image is downloaded and operational on all nodes.

Once the new image has been downloaded and started on all nodes you should see something like:

In [48]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE              ERROR               PORTS
vqet7sf9g2yr        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 22 seconds ago                         
9zxra8gie7yc        docker-demo.2       mjbright/docker-demo:21   swnode1             Running             Preparing 2 seconds ago                        
eqijlhjhrxd7         \_ docker-demo.2   mjbright/docker-demo:20   swnode1             Shutdown            Shutdown 1 second ago                          
hksw0wxtx4il        docker-demo.3       mjbright/docker-demo:20   swnode2             Running             Preparing 10 seconds ago                       


### Verifying the service has been updated

Then open your web browser at the page http://localhost:8080 and you should now see a lovely **red** whale.


![](images/docker_red.png)

of if you prefer the command-line ..

In [49]:
NB_docker_loop_until_N_replicas_started docker-demo 3

#NB_pause

1/3
... waiting for 'docker-demo' 3 replicas to start
1/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-

In [50]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE
jusq9inajzd1        docker-demo         replicated          3/3                 mjbright/docker-demo:21


In [51]:
NB_docker_loop_until_service_started docker-demo

jusq9inajzd1        docker-demo         replicated          3/3                 mjbright/docker-demo:21


In [52]:
wget -O - http://$MASTERIP:8080

--2017-05-13 20:14:28--  http://192.168.99.114:8080/
Connecting to 192.168.99.114:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/txt]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
          

# drain a node

We can drain a node effectively placing it in 'maintenance mode'.

Draining a node means that it no longer has running tasks on it.

In [53]:
docker $(docker-machine config swmaster1) node ls

ID                           HOSTNAME   STATUS  AVAILABILITY  MANAGER STATUS
m5i0938m512yeqtj2vgwoss38 *  swmaster1  Ready   Active        Leader
obms842spsey4of2hfp795pa0    swnode2    Ready   Active        
s6eekxu2v21eyysjeelerkfrw    swnode1    Ready   Active        


Let's drain swnode1

In [54]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                 ERROR               PORTS
kh1k6uccp1kr        docker-demo.1       mjbright/docker-demo:21   swmaster1           Running             Running 11 seconds ago                            
vqet7sf9g2yr         \_ docker-demo.1   mjbright/docker-demo:20   swmaster1           Shutdown            Shutdown about a minute ago                       
9zxra8gie7yc        docker-demo.2       mjbright/docker-demo:21   swnode1             Running             Running 3 minutes ago                             
eqijlhjhrxd7         \_ docker-demo.2   mjbright/docker-demo:20   swnode1             Shutdown            Shutdown 4 minutes ago                            
hk5o4ruqtc3j        docker-demo.3       mjbright/docker-demo:21   swnode2             Running             Running about a minute ago                        
hksw0wxtx4il         \_ docker-demo.3   mjbright/dock

In [55]:
docker $(docker-machine config swmaster1) node update --availability drain swnode1

swnode1


and now we see that all services on swnode1 are shutdown

In [56]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                  ERROR               PORTS
kh1k6uccp1kr        docker-demo.1       mjbright/docker-demo:21   swmaster1           Running             Running 14 seconds ago                             
vqet7sf9g2yr         \_ docker-demo.1   mjbright/docker-demo:20   swmaster1           Shutdown            Shutdown about a minute ago                        
zlqka7ebxj15        docker-demo.2       mjbright/docker-demo:21   swmaster1           Ready               Ready less than a second ago                       
9zxra8gie7yc         \_ docker-demo.2   mjbright/docker-demo:21   swnode1             Shutdown            Running 1 second ago                               
eqijlhjhrxd7         \_ docker-demo.2   mjbright/docker-demo:20   swnode1             Shutdown            Shutdown 4 minutes ago                             
hk5o4ruqtc3j        docker-demo.3       mjbrigh

But as we had request 3 replicas of our service the Swarm mode will take care to start extra instances on other nodes so that we still have 3 replicas of the service running.

So after a few seconds we see that we once again have 3 replicas in the *Running* state, as shown below:

In [57]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                 ERROR               PORTS
kh1k6uccp1kr        docker-demo.1       mjbright/docker-demo:21   swmaster1           Running             Running 15 seconds ago                            
vqet7sf9g2yr         \_ docker-demo.1   mjbright/docker-demo:20   swmaster1           Shutdown            Shutdown about a minute ago                       
zlqka7ebxj15        docker-demo.2       mjbright/docker-demo:21   swmaster1           Ready               Ready 2 seconds ago                               
9zxra8gie7yc         \_ docker-demo.2   mjbright/docker-demo:21   swnode1             Shutdown            Running 2 seconds ago                             
eqijlhjhrxd7         \_ docker-demo.2   mjbright/docker-demo:20   swnode1             Shutdown            Shutdown 4 minutes ago                            
hk5o4ruqtc3j        docker-demo.3       mjbright/dock

# remove a service

Now let's cleanup by removing our service

In [58]:
docker $(docker-machine config swmaster1) service rm docker-demo

docker-demo


We can check that the service is no longer running:

In [59]:
docker $(docker-machine config swmaster1) service ps docker-demo

NB_continue

no such services: docker-demo


In [60]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE                     COMMAND                  CREATED             STATUS              PORTS               NAMES
c6e072f0ce71        mjbright/docker-demo:21   "/bin/sh -c 'pytho..."   21 seconds ago      Up 19 seconds                           docker-demo.1.kh1k6uccp1kr1hv6fee152xkx


In [61]:
NB_time_taken

Started at Sat May 13 20:14:38 CEST 2017   [1494698568]
Ended   at Sat May 13 20:14:38 CEST 2017   [1494699278]
Took 710 secs [0h11m50]
